In [ ]:
#imports
import pandas as pd
from sqlalchemy import create_engine, select, func, MetaData, Table, Column, Integer, String
from dotenv import load_dotenv
import os

load_dotenv()

user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
name = os.getenv("DB_NAME")
port = 3306

cols = ['posteam', 'play_type', 'yards_gained', 'pass_attempt', 'rush_attempt', 'pass_touchdown', 'rush_touchdown']
data = pd.read_csv('reg_pbp_2009.csv', usecols = cols)
cleaned_data = data.dropna()

In [55]:
cleaned_data['pass_yards'] = cleaned_data['yards_gained'] * cleaned_data['pass_attempt']
cleaned_data['rush_yards'] = cleaned_data['yards_gained'] * cleaned_data['rush_attempt']

stats = (cleaned_data.groupby('posteam', as_index = False)
        .agg(
            pass_yards = ('pass_yards', 'sum'),
            rush_yards = ('rush_yards', 'sum'),
            pass_attempts = ('pass_attempt', 'sum'),
            rush_attempts = ('rush_attempt', 'sum'),
            pass_touchdowns = ('pass_touchdown', 'sum'),
            rush_touchdowns = ('rush_touchdown', 'sum'),
            )
        )

#stats

In [56]:
try:
    engine_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{name}"
    engine = create_engine(engine_string)
    meta = MetaData()
except Exception as e:
    print("Error while connecting to MySQL:", e)

In [57]:


team_stats = Table('2009_team_stats', meta, Column('posteam', String(255)), 
                   Column('pass_yards',Integer), Column('rush_yards', Integer),
                   Column('pass_attempts', Integer), Column('rush_attempts', Integer),
                   Column('pass_touchdowns', Integer), Column('rush_touchdowns', Integer)
    )

meta.create_all(engine)
    

In [58]:
stats.to_sql(name = '2009_team_stats', con = engine, if_exists = 'replace',index = False)



32

In [59]:
query = select(func.sum(team_stats.c.pass_attempts))

with engine.connect() as conn:
    result = conn.execute(query).scalar()
    print(f"2009 total pass attempts: {result}")
    

2009 total pass attempts: 18178.0


In [ ]:
query = select(func.sum(team_stats.c.rush_attempts))

with engine.connect() as conn:
    result = conn.execute(query).scalar()
    print(f"2009 total rush attempts: {result}")

2009 total pass attempts: 14104.0
